In [ ]:
from google.colab import drive
import pandas as pd
from tqdm import tqdm

drive.mount('/content/drive')
path = '/content/drive/MyDrive/6000_IMDB_Dataset.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import BertTokenizer, BertModel
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from transformers import AdamW
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

review_df = pd.read_csv(path)
feature = review_df['review'].tolist()
label = review_df['sentiment'].tolist()

for i in range(len(label)):
    if label[i] == 'positive':
        label[i] = 1
    else :
        label[i] = 0

train_texts,test_texts, train_labels,test_labels = train_test_split(feature, label, train_size=0.7, shuffle=True)

# 토큰화
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt')

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
## 기존 bertforseqclassification
from transformers import BertPreTrainedModel
from transformers import PreTrainedModel
from transformers import BertConfig
from typing import List, Optional, Tuple, Union
from transformers.utils import (
    add_code_sample_docstrings,
    add_start_docstrings,
    add_start_docstrings_to_model_forward,
    get_torch_version,
    replace_return_docstrings,
    ModelOutput,
    logging
)

from transformers.modeling_outputs import (
    BaseModelOutputWithPastAndCrossAttentions,
    BaseModelOutputWithPoolingAndCrossAttentions,
    CausalLMOutputWithCrossAttentions,
    MaskedLMOutput,
    MultipleChoiceModelOutput,
    NextSentencePredictorOutput,
    QuestionAnsweringModelOutput,
    SequenceClassifierOutput,
    TokenClassifierOutput,
)

from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        self.bert = BertModel(config)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        # Initialize weights and apply final processing
        self.post_init()

    """
    @add_start_docstrings_to_model_forward(BERT_INPUTS_DOCSTRING.format("batch_size, sequence_length"))
    @add_code_sample_docstrings(
        checkpoint=_CHECKPOINT_FOR_SEQUENCE_CLASSIFICATION,
        output_type=SequenceClassifierOutput,
        config_class=_CONFIG_FOR_DOC,
        expected_output=_SEQ_CLASS_EXPECTED_OUTPUT,
        expected_loss=_SEQ_CLASS_EXPECTED_LOSS,
    )
    """
    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], SequenceClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)
        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

# 사전 훈련된 BERT 모델 이름 설정
config = BertConfig.from_pretrained('bert-base-uncased')
print(config)
config = BertConfig(
    num_labels=2,
    hidden_dropout_prob=0.2,
    classifier_dropout=0.1,
)
"""
default
"attention_probs_dropout_prob": 0.1,
"classifier_dropout": null,
"gradient_checkpointing": false,
"hidden_act": "gelu",
"hidden_dropout_prob": 0.1,
"hidden_size": 768,
"initializer_range": 0.02,
"intermediate_size": 3072,
"layer_norm_eps": 1e-12,
"max_position_embeddings": 512,
"model_type": "bert",
"num_attention_heads": 12,
"num_hidden_layers": 12,
"pad_token_id": 0,
"position_embedding_type": "absolute",
"transformers_version": "4.40.2",
"type_vocab_size": 2,
"use_cache": true,
"vocab_size": 30522
"""


# 새로운 설정으로 모델 초기화
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.40.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [ ]:
# 훈련 설정
optimizer = AdamW(model.parameters(), lr=5e-5)

# 데이터셋 생성
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)

epoch_num = 1
model.train()
for epoch in range(epoch_num):
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        print(outputs.logits)
        print(labels)
        print(torch.argmax(outputs.logits, dim=1))
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1)
        total += labels.size(0)
        correct += (predicted_labels == labels).sum().item()


accuracy = correct / total
print(f'테스트 정확도: {accuracy}')


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/525 [00:00<?, ?it/s]

tensor([[ 0.1026, -0.0021],
        [-0.0208, -0.0363],
        [-0.0441,  0.0315],
        [ 0.1712, -0.2988],
        [ 0.1308, -0.1072],
        [ 0.0385, -0.0526],
        [ 0.2350, -0.0773],
        [ 0.0065, -0.0692]], grad_fn=<AddmmBackward0>)
tensor([1, 1, 0, 1, 1, 1, 1, 0])
tensor([0, 0, 1, 0, 0, 0, 0, 0])


  0%|          | 1/525 [00:03<29:49,  3.41s/it]

tensor([[-0.1757,  0.0033],
        [-0.0155,  0.4972],
        [-0.0168,  0.3541],
        [ 0.0242,  0.3053],
        [-0.0098,  0.3098],
        [-0.0557,  0.2992],
        [-0.0482,  0.1509],
        [ 0.0113,  0.2492]], grad_fn=<AddmmBackward0>)
tensor([0, 1, 1, 0, 0, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1])


  0%|          | 2/525 [00:06<30:26,  3.49s/it]

...

  4%|▍         | 20/525 [01:03<26:35,  3.16s/it]

tensor([[ 0.0024, -0.1996],
        [ 0.0230, -0.1440],
        [-0.2088, -0.2075],
        [-0.2448,  0.0097],
        [-0.2344, -0.2221],
        [-0.1854,  0.0911],
        [-0.2900, -0.3012],
        [-0.1005,  0.3007]], grad_fn=<AddmmBackward0>)
tensor([1, 0, 0, 1, 0, 1, 0, 1])
tensor([0, 0, 1, 1, 1, 1, 0, 1])


  4%|▍         | 21/525 [01:06<26:33,  3.16s/it]

tensor([[-0.0905,  0.1394],
        [-0.3247,  0.1456],
        [-0.3581, -0.0920],
        [-0.1321, -0.1669],
        [ 0.2071, -0.0502],
        [-0.1291, -0.5646],
        [ 0.0237,  0.1763],
        [-0.1041, -0.2020]], grad_fn=<AddmmBackward0>)
tensor([0, 1, 0, 1, 1, 1, 1, 0])
tensor([1, 1, 1, 0, 0, 0, 1, 0])


  4%|▍         | 22/525 [01:09<27:07,  3.24s/it]